In [7]:
import docplex
from docplex.mp.model import Model
import numpy as np

In [153]:
import time
import math

class ContextTimer(object):
    def __init__(self, msg):
        self.msg = msg
        self.start = 0
        
    def __enter__(self):
        self.start = time.time()
        print('--> begin {0}'.format(self.msg))
        return self  # return value is value of with ()
        
    def __exit__(self, *args):
        elapsed = time.time() - self.start
        self.msecs = math.ceil(1000* elapsed)
        print('<-- end {0},  time: {1:.0f} ms'.format(self.msg, self.msecs))   
        
# try our timer on computing fibonacci numbers
def fib(n):
    return 1 if n <= 2 else  fib(n-1) + fib(n-2)

# timing fibonacci(30)
with ContextTimer("fibonacci 30"):
    n = 30
    f = fib(n)
    print("fibonacci({0}) = {1}".format(n, f))
        

--> begin fibonacci 30
fibonacci(30) = 832040
<-- end fibonacci 30,  time: 371 ms


In [232]:
def gali_bids_init(value_model, M, n):
    np.random.seed(42)
    models = []
    #M = 5  # number of items in value model
    #n = 5 # number of bundles to be sampled
    b_0 = np.asarray(np.random.choice([0, 1], size=M))
    S_matrix = []
    S_matrix.append(b_0)
    for i in range(n-1): #start with next bundle
        n_S_matrix = len(S_matrix)
        x_vectors = []
        x_distances = []
        for j in range(n_S_matrix): # solve a MIP for each bundle
            m = Model(name='greedy sampling')
            indices = range(M)
            x = m.binary_var_list(keys = M, name='x')

            #Define objective function
            obj_function = m.sum(x[i] + S_matrix[j][i] - 2 * x[i] * S_matrix[j][i] for i in indices)

            
            for k in range(n_S_matrix):
                if k == j:
                    continue
                constraint_expr = m.sum(x[i] + S_matrix[k][i] - 2 * x[i] * S_matrix[k][i] for i in indices)
                m.add_constraint(obj_function <= constraint_expr)

            m.maximize(obj_function)
            m.solve()
            models.append(m)
            vector = np.array([x[i].solution_value for i in range(M)])
            #Calculate manhattan distance between bundle and corresponding bundle in S_matrix
            distances = np.sum(np.abs(vector - S_matrix[j]))            
            x_vectors.append(vector)
            x_distances.append(distances)
        #print('All found vectors')
        #print(x_vectors)
        S_matrix.append(x_vectors[np.argmax(x_distances)])
        #print('We append this vector to S_matrix')
        #print(x_vectors[np.argmax(x_distances)])
    D = np.array(S_matrix)
    return (D), models

In [255]:
def gali_bids_init_new(value_model, M, n):
    np.random.seed(42)
    models = []
    #M = 5  # number of items in value model
    #n = 5 # number of bundles to be sampled
    b_0 = np.asarray(np.random.choice([0, 1], size=M))
    S_matrix = []
    S_matrix.append(b_0)
    for i in range(n-1): #start with next bundle
        n_S_matrix = len(S_matrix)
        x_vectors = []
        x_distances = []
        for j in range(n_S_matrix): # solve a MIP for each bundle
            m = Model(name='greedy sampling')
            indices = range(M)
            x = m.binary_var_list(keys = M, name='x')

            #Define objective function
            obj_function = m.sum(x[i] + S_matrix[j][i] - 2 * x[i] * S_matrix[j][i] for i in indices)

            constraints = []
            for k in range(n_S_matrix):
                if k == j:
                    continue
                constraint_expr = m.sum(x[i] + S_matrix[k][i] - 2 * x[i] * S_matrix[k][i] for i in indices)
                constraints.append(obj_function <= constraint_expr)
            m.add_constraints(constraints)

            m.maximize(obj_function)
            m.solve()
            models.append(m)

            vector = np.array([x[i].solution_value for i in range(M)])
            #Calculate manhattan distance between bundle and corresponding bundle in S_matrix
            distances = np.sum(np.abs(vector - S_matrix[j]))            
            x_vectors.append(vector)
            x_distances.append(distances)
        #print('All found vectors')
        #print(x_vectors)
        S_matrix.append(x_vectors[np.argmax(x_distances)])
        #print('We append this vector to S_matrix')
        #print(x_vectors[np.argmax(x_distances)])
    D = np.array(S_matrix)
    return (D), models

In [257]:
#set seed
#plot results
np.random.seed(42)
sizes = [5, 10]
init = []
init_new = []
for sz in sizes:
    with ContextTimer("bench1_size_1000"):
        output, model = gali_bids_init(None,10,sz)
    #model.print_information()
    init.append(model)
    with ContextTimer("bench2_size_1000"):
        output, n_model = gali_bids_init_new(None, 10, sz)
    init_new.append(n_model)
    # model.print_information()
    # print(output)


--> begin bench1_size_1000
<-- end bench1_size_1000,  time: 2908 ms
--> begin bench2_size_1000
<-- end bench2_size_1000,  time: 3369 ms
--> begin bench1_size_1000
<-- end bench1_size_1000,  time: 15056 ms
--> begin bench2_size_1000
<-- end bench2_size_1000,  time: 12119 ms


In [258]:
try:
    import matplotlib.pyplot as plt
    %matplotlib inline
except ImportError:
    print("try install matplotlib: pip install matplotlib")
    raise

In [267]:
# plt.figure(figsize=(16,7))
# plt.plot(sizes, init[0], label="one_by_one")
# plt.plot(sizes, init_new[0], label="var_list")
# plt.legend()
# plt.show()
#import numpy as np
#calculate runtime
#calculate mean runtime
init_runtime = []
for each in init:
    for i in range(len(each)):
        init_runtime.append(each[i].solve_details.time)
init_new_runtime = []
for each in init_new:
    for i in range(len(each)):
        init_new_runtime.append(each[i].solve_details.time)
#print(np.mean(init_new)[0])
#print(np.mean(init)[0])


In [268]:
import numpy as np
print(np.mean(init_runtime))
print(np.mean(init_new_runtime))

0.299120890010487
0.26301933201876554


In [116]:
#Calculate runtime
import time
start_time = time.time()
arr = []
for i in range(1000):
    arr.append(1)
np_arr = np.array(arr)
print("--- %s seconds ---" % (time.time() - start_time))
#print(np_arr)

--- 0.0004203319549560547 seconds ---


In [131]:
#Calculate runtime
import time
start_time = time.time()
n_arr = np.array([1,2,3])
for i in range(1000):
    np.append(n_arr, [i], axis=0)
print("--- %s seconds ---" % (time.time() - start_time))
print(n_arr)

--- 0.007542610168457031 seconds ---
[1 2 3]
